In [1]:
import rdkit

In [2]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors as desc
from rdkit.DataStructs import cDataStructs as ds
from rdkit.DataStructs import UIntSparseIntVect as usv
from rdkit.Chem import MACCSkeys, DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Pairs

In [56]:
import csv

In [3]:
import rdkit.Chem.Fingerprints
fulltrain = pd.read_csv("orig_train.csv")
fulltest = pd.read_csv("orig_test.csv")

In [1]:
fulltest.shape

NameError: name 'fulltest' is not defined

In [4]:
train_smiles = fulltrain['smiles']
test_smiles = fulltest['smiles']

In [102]:
testId = fulltest['Id']

In [40]:
num_comparers = 500
train_size = 20000
valid_size = 10000
comparers = train_smiles[:num_comparers]
ytrain = fulltrain['gap']
ytrain1 = ytrain[num_comparers:(num_comparers+train_size)]
yvalid = ytrain[(num_comparers+train_size):(num_comparers+train_size+valid_size)]

In [ ]:
comp_morgans = []
for comp in comparers:
    m = Chem.MolFromSmiles(comp)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    comp_morgans.append(fp)

In [91]:
fopen = open("test_sims_all.csv", 'wb')
transform_csv = csv.writer(fopen, delimiter=',', quotechar='"')
test_sims = pd.DataFrame()
prog = 0

for smile in test_smiles: 
    smile_sim = []
    prog+= 1
    if prog%10000==0:
        print prog
    m = Chem.MolFromSmiles(smile)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    for comp in comp_morgans:
        sim = DataStructs.DiceSimilarity(comp,fp)
        smile_sim.append(sim)
    transform_csv.writerow(smile_sim)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000


In [98]:
test_simes_all = pd.read_csv("test_sims_all.csv",header=None)

In [115]:
test_simes_all.loc[824229,320:500].isnull()

320    False
321    False
322    False
323    False
324    False
325    False
326    False
327     True
328     True
329     True
330     True
331     True
332     True
333     True
334     True
335     True
336     True
337     True
338     True
339     True
340     True
341     True
342     True
343     True
344     True
345     True
346     True
347     True
348     True
349     True
       ...  
470     True
471     True
472     True
473     True
474     True
475     True
476     True
477     True
478     True
479     True
480     True
481     True
482     True
483     True
484     True
485     True
486     True
487     True
488     True
489     True
490     True
491     True
492     True
493     True
494     True
495     True
496     True
497     True
498     True
499     True
Name: 824229, dtype: bool

In [99]:
import os
os.system('say "your program has finished"')

0

In [89]:
def getMols(transform_filename, data):
    
    tf = open(transform_filename, 'wb')

    # Produce a CSV File:
    transform_csv = csv.writer(tf, delimiter=',', quotechar='"')

    prog = 0
    for index, datum in enumerate(data):
        prog += 1
        if prog % 10000 == 0:
            print prog
        m = Chem.MolFromSmiles(datum)
        fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
        transform_csv.writerow(np.hstack((
                                datum,
                                m,
                                fp,
                                ))
                               )
    print "done"

In [ ]:
tf = open(transform_filename, 'wb')

# Produce a CSV File:
transform_csv = csv.writer(tf, delimiter=',', quotechar='"')

prog = 0
for index, datum in enumerate(data):
    prog += 1
    if prog % 10000 == 0:
        print prog
    m = Chem.MolFromSmiles(datum)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    transform_csv.writerow(np.hstack((datum,m,fp,)))
print "done"

In [61]:
getMols("test_mols_complete.csv",test_smiles)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
done


In [62]:
import os
os.system('say "your program has finished"')

0

In [ ]:
mols_test = pd.read_csv('test_mols_complete.csv',names=["smile","mol"])

In [84]:
mol_lst = []
for i in range(mols_test.shape[0]):
    mol_lst.append(mols_test['mol'][i])

In [88]:
type(mol_lst[1])

str

In [59]:
def getFeatures(transform_filename, data):
    
    tf = open(transform_filename, 'wb')

    # Produce a CSV File:
    transform_csv = csv.writer(tf, delimiter=',', quotechar='"')

    prog = 0
    for index, datum in enumerate(data):
        prog += 1
        if prog % 10000 == 0:
            print prog
        transform_csv.writerow(np.hstack((
                                datum,
                                AllChem.GetMorganFingerprintAsBitVect(datum,2,useFeatures=True),
                                ))
                               )
    print "done"

In [87]:

getFeatures("test_morgan_complete.csv",mol_lst)


ArgumentError: Python argument types in
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(str, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None)

In [41]:
comp_morgans = []
for comp in comparers:
    m = Chem.MolFromSmiles(comp)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    comp_morgans.append(fp)


In [80]:
for comp in comparers[0]:
    m = Chem.MolFromSmiles(comp)
    print type(m)

<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>
<type 'NoneType'>


In [42]:
fps = []
for smile in train_smiles[num_comparers:(num_comparers+train_size)]: 
    m = Chem.MolFromSmiles(smile)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    fps.append(fp)


In [43]:
fps_valid = []
for smile in train_smiles[(num_comparers+train_size):(num_comparers+train_size+valid_size)]: 
    m = Chem.MolFromSmiles(smile)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    fps_valid.append(fp)


In [44]:
train_sims = pd.DataFrame()
for comp in comp_morgans:
    comp_sim = []
    for fp in fps:
        sim = DataStructs.DiceSimilarity(comp,fp)
        comp_sim.append(sim)
    train_sims[comp] = comp_sim

In [45]:
valid_sims = pd.DataFrame()
for comp in comp_morgans:
    comp_sim = []
    for fp in fps_valid:
        sim = DataStructs.DiceSimilarity(comp,fp)
        comp_sim.append(sim)
    valid_sims[comp] = comp_sim

In [46]:
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=100)
rf.fit(train_sims,ytrain1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [25]:
rf2 = sklearn.ensemble.RandomForestRegressor(n_estimators=100)
rf2.fit(train_sims,ytrain1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [119]:
test_simes_all.loc[824225:,326:500].isnull()

,326,327,328,329,330,331,332,333,334,335,...,490,491,492,493,494,495,496,497,498,499
824225,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
824226,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
824227,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
824228,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
824229,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [121]:
preds_out = pd.DataFrame(testId)
preds_test = rf.predict(test_simes_all.loc[:824228,])

#mean_squared_error(yvalid,preds)

In [122]:
import os
os.system('say "your program has finished"')

0

In [135]:
print len(preds_test)
print preds_out.shape
new = np.append(preds_test, preds_test.mean())

824229
(824230, 1)


In [136]:
len(new)

824230

In [138]:
preds_out['Prediction']=new

In [141]:
preds_out

,Id,Prediction
0,1,1.883700
1,2,1.882727
2,3,1.650833
3,4,1.671800
4,5,1.768780
5,6,2.256350
6,7,2.070020
7,8,1.756467
8,9,1.313733
9,10,2.230000


In [142]:
preds_out.to_csv("similarity_preds.csv",index=None)

In [26]:
preds2 = rf2.predict(valid_sims) 
mean_squared_error(yvalid,preds2)

0.04428751999465233

In [32]:
train_sims.columns=[range(train_sims.shape[1])]
train_sims

,0,1,2,3,4,5,6,7,8,9,...,489,490,491,492,493,494,495,496,497,498
0,0.437500,0.405405,0.307692,0.493151,0.333333,0.384615,0.474576,0.579710,0.531250,0.422535,...,0.422535,0.350877,0.843750,0.309859,0.406780,0.376812,0.612903,0.363636,0.352941,0.361111
1,0.347826,0.455696,0.385542,0.589744,0.507042,0.361446,0.437500,0.621622,0.318841,0.500000,...,0.526316,0.516129,0.405797,0.394737,0.375000,0.378378,0.298507,0.394366,0.465753,0.467532
2,0.408163,0.237288,0.317460,0.275862,0.509804,0.380952,0.409091,0.333333,0.367347,0.321429,...,0.357143,0.476190,0.244898,0.321429,0.318182,0.333333,0.340426,0.352941,0.415094,0.315789
3,0.527778,0.390244,0.418605,0.419753,0.378378,0.465116,0.388060,0.389610,0.361111,0.556962,...,0.405063,0.338462,0.388889,0.379747,0.238806,0.337662,0.371429,0.351351,0.526316,0.475000
4,0.465753,0.385542,0.551724,0.390244,0.293333,0.620690,0.323529,0.384615,0.410959,0.425000,...,0.425000,0.242424,0.438356,0.350000,0.323529,0.589744,0.535211,0.346667,0.285714,0.469136
5,0.476190,0.328767,0.389610,0.361111,0.430769,0.649351,0.448276,0.617647,0.507937,0.514286,...,0.514286,0.428571,0.349206,0.285714,0.241379,0.441176,0.459016,0.461538,0.417910,0.788732
6,0.350877,0.447761,0.394366,0.515152,0.338983,0.338028,0.346154,0.516129,0.315789,0.593750,...,0.437500,0.480000,0.456140,0.343750,0.461538,0.387097,0.290909,0.474576,0.491803,0.430769
7,0.596491,0.298507,0.309859,0.303030,0.474576,0.366197,0.576923,0.580645,0.526316,0.281250,...,0.468750,0.600000,0.596491,0.437500,0.230769,0.483871,0.581818,0.406780,0.360656,0.461538
8,0.535211,0.493827,0.376471,0.425000,0.301370,0.447059,0.333333,0.526316,0.450704,0.410256,...,0.435897,0.343750,0.450704,0.307692,0.333333,0.500000,0.463768,0.493151,0.480000,0.481013
9,0.333333,0.315789,0.350000,0.320000,0.411765,0.500000,0.393443,0.394366,0.484848,0.575342,...,0.657534,0.440678,0.272727,0.301370,0.229508,0.281690,0.343750,0.500000,0.428571,0.675676


In [36]:
valid_sims.columns=[range(train_sims.shape[1])]

In [37]:
xgtrain = xgb.DMatrix(train_sims, label=ytrain1)
xgtest = xgb.DMatrix(valid_sims,yvalid)

param = {'max_depth':100, 'eta':1, 'silent':0, 'objective':'reg:linear','learning_rate':0.6}
num_round = 10
xg = xgb.train(param, xgtrain, num_round)
#cvresult = xgb.cv(param, xgtrain, num_boost_round=1000, nfold=5, metrics='auc', early_stopping_rounds=50)

preds_xg = xg.predict(xgtest)
error = mean_squared_error(yvalid,preds_xg)
error


0.065275557046950344

In [39]:
avg = (preds_xg+preds+preds2)/3
mean_squared_error(yvalid,avg)

0.045286208647584474

In [49]:
# Create first network with Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
#dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = train_sims
Y = ytrain1
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10)
# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


Using TensorFlow backend.


ImportError: No module named google.protobuf

In [ ]:
import os
os.system('say "your program has finished"')

In [3]:
import pandas as pd
pd.__version__

u'0.17.1'